In [1]:
## Codigo Principal
import os
import pandas as pd
import win32com.client
from datetime import datetime, timedelta

def processar_aniversario_empresa():
    # === CONFIGURAÇÕES ===
    CAMINHO_EXCEL = r"C:\Users\lz6321\OneDrive - grendene.com.br\Documentos\automation\Automação_Aniversariantes\Aniversario - Empresa.xlsx"
    NOME_ABA = "Sheet1"

    MES_PROCESSAR = 8  # Agosto
    ANO = datetime.now().year

    MESES_ABREV = {
        1: "Jan", 2: "Feb", 3: "Mar", 4: "Apr", 5: "May", 6: "Jun",
        7: "Jul", 8: "Aug", 9: "Sep", 10: "Oct", 11: "Nov", 12: "Dec"
    }

    PASTA_CARDS = r"C:\Users\lz6321\OneDrive - grendene.com.br\Documentos\automation\Automação_Aniversariantes\Aniversariantes - Março"
    CAMINHO_MARCA = r"C:\Users\lz6321\OneDrive - grendene.com.br\Documentos\automation\Automação_Aniversariantes\Marca_GovTi.png"
    EMAIL_DESTINO = "carla.vieira@grendene.com.br"

    try:
        df = pd.read_excel(CAMINHO_EXCEL, sheet_name=NOME_ABA)

        df = df[df['aniversario_empresa MÊS'].str.strip().str.capitalize() == MESES_ABREV[MES_PROCESSAR]]

        if df.empty:
            print(f"[INFO] Não há aniversariantes de empresa no mês {MESES_ABREV[MES_PROCESSAR]}")
            return

        df = df.dropna(subset=['aniversario_empresa DIA'])
        df['aniversario_empresa DIA'] = df['aniversario_empresa DIA'].astype(int)

        outlook = win32com.client.Dispatch("Outlook.Application")
        arquivos_cartoes = os.listdir(PASTA_CARDS) if os.path.exists(PASTA_CARDS) else []

        def formatar_tempo(t):
            return f"{t} ano" if t == 1 else f"{t} anos"

        for dia, grupo in df.groupby('aniversario_empresa DIA'):
            nomes = grupo['Nome Funcionário'].tolist()
            tempos = grupo['Tempo Empresa'].astype(int).tolist()

            if not nomes:
                continue

            # Horário fixo 08:00 às 08:30 (ajuste de fuso se necessário)
            data_evento = datetime(ANO, MES_PROCESSAR, int(dia), 8, 0) - timedelta(hours=3)

            nomes_txt = "\n".join(f"{nome}, {formatar_tempo(tempo)};" for nome, tempo in zip(nomes, tempos))
            nomes_html = "<br>".join(f"<b>{nome}</b>, {formatar_tempo(tempo)};" for nome, tempo in zip(nomes, tempos))

            print(f"\n[PROCESSANDO] Aniversário de empresa para {nomes_txt}")

            # === Definição do título ===
            if len(nomes) == 1:
                titulo_evento = f"Aniversário de Empresa do Colaborador - {nomes[0]}"
                titulo_email = f"Aniversário de Empresa do Colaborador - {nomes[0]}"
                palavra_colaborador = "colaborador"
            else:
                titulo_evento = f"Aniversário de Empresa dos Colaboradores - {', '.join(nomes)}"
                titulo_email = f"Aniversário de Empresa dos Colaboradores - {', '.join(nomes)}"
                palavra_colaborador = "colaboradores"

            # === Evento ===
            appointment = outlook.CreateItem(1)
            appointment.Subject = titulo_evento
            appointment.Start = data_evento
            appointment.End = data_evento + timedelta(minutes=30)
            appointment.AllDayEvent = False
            appointment.ReminderSet = True
            appointment.ReminderMinutesBeforeStart = 1440
            appointment.BusyStatus = 0

            corpo_evento = f"""Olá!
Segue lembrete de aniversário de empresa para o(s) {palavra_colaborador}:
{nomes_txt}

ATENÇÃO!: No dia do aniversário de empresa do(s) referido(s) {palavra_colaborador} deve-se enviar a imagem em anexo na equipe de celebrações no Teams.

Atenciosamente,"""
            appointment.Body = corpo_evento

            for nome in nomes:
                nome_lower = nome.lower()
                for arquivo in arquivos_cartoes:
                    if nome_lower in arquivo.lower():
                        appointment.Attachments.Add(os.path.join(PASTA_CARDS, arquivo))
                        break

            appointment.Display()

            # === Email ===
            mail = outlook.CreateItem(0)
            mail.Subject = titulo_email
            mail.To = EMAIL_DESTINO

            for nome in nomes:
                nome_lower = nome.lower()
                for arquivo in arquivos_cartoes:
                    if nome_lower in arquivo.lower():
                        mail.Attachments.Add(os.path.join(PASTA_CARDS, arquivo))
                        break

            img_tag = ""
            if os.path.exists(CAMINHO_MARCA):
                attachment = mail.Attachments.Add(CAMINHO_MARCA)
                attachment.PropertyAccessor.SetProperty(
                    "http://schemas.microsoft.com/mapi/proptag/0x3712001F", "marca"
                )
                img_tag = '<p><img src="cid:marca" style="width:300px;"></p>'

            corpo_email = f"""
<html>
<body>
<p>Olá!</p>
<p>Segue lembrete de aniversário de empresa para o(s) {palavra_colaborador}:<br>
{nomes_html}</p>
<p><b>ATENÇÃO!:</b> No dia do aniversário de empresa do(s) referido(s) {palavra_colaborador} deve-se enviar a imagem em anexo na equipe de celebrações no Teams.</p>
<p>Atenciosamente,</p>
{img_tag}
</body>
</html>
"""
            mail.HTMLBody = corpo_email

            mail.Display()
            print(f"[OK] Evento e email preparados para {', '.join(nomes)}")

        print("\n[FINALIZADO] Processamento concluído!")

    except Exception as e:
        print(f"[ERRO] Falha geral na automação: {str(e)}")

if __name__ == "__main__":
    processar_aniversario_empresa()


[PROCESSANDO] Aniversário de empresa para DOUGLAS AGUIAR ALCANTARA, 1 ano;
MILENA STEMPKOSKI, 2 anos;
LAURA NALLEM FRANCISQUETTI, 2 anos;
ANTONIO HELDER SANTIAGO MOREIRA, 2 anos;
[OK] Evento e email preparados para DOUGLAS AGUIAR ALCANTARA, MILENA STEMPKOSKI, LAURA NALLEM FRANCISQUETTI, ANTONIO HELDER SANTIAGO MOREIRA

[PROCESSANDO] Aniversário de empresa para GIRLEI DIAS, 4 anos;
[OK] Evento e email preparados para GIRLEI DIAS

[PROCESSANDO] Aniversário de empresa para TAILOR PIUCO, 5 anos;
[OK] Evento e email preparados para TAILOR PIUCO

[PROCESSANDO] Aniversário de empresa para NAIARA DA SILVA CAMARGO, 1 ano;
LUCAS WEBBER MOLIN, 1 ano;
DENER DAVI SOUSA LIMA, 3 anos;
[OK] Evento e email preparados para NAIARA DA SILVA CAMARGO, LUCAS WEBBER MOLIN, DENER DAVI SOUSA LIMA

[PROCESSANDO] Aniversário de empresa para CAMILA DE AZEVEDO OLIVEIRA, 2 anos;
[OK] Evento e email preparados para CAMILA DE AZEVEDO OLIVEIRA

[PROCESSANDO] Aniversário de empresa para JOSE ANDERSON PONTE PRADO, 3 ano

In [ ]:
## Codigo Teste
import os
import pandas as pd
import win32com.client
from datetime import datetime


def processar_aniversario_empresa():
    # === CONFIGURAÇÕES ===
    CAMINHO_EXCEL = r"C:\Users\lz6321\OneDrive - grendene.com.br\Documentos\automation\Automação_Aniversariantes\Aniversariante - Empresa.xlsx"
    NOME_ABA = "Sheet1"

    MES_CARDS = 3  # Março (pasta dos cartões)
    MES_PROCESSAR = 8  # Agosto (mês do evento)
    ANO = datetime.now().year

    MESES_ABREV = {
        1: "Jan", 2: "Feb", 3: "Mar", 4: "Apr", 5: "May", 6: "Jun",
        7: "Jul", 8: "Aug", 9: "Sep", 10: "Oct", 11: "Nov", 12: "Dec"
    }

    PASTA_CARDS = r"C:\Users\lz6321\OneDrive - grendene.com.br\Documentos\automation\Automação_Aniversariantes\Aniversariantes - Março"
    CAMINHO_MARCA = r"C:\Users\lz6321\OneDrive - grendene.com.br\Documentos\automation\Automação_Aniversariantes\Marca_GovTi.png"
    EMAIL_DESTINO = "carla.vieira@grendene.com.br"

    try:
        df = pd.read_excel(CAMINHO_EXCEL, sheet_name=NOME_ABA)

        mes_abrev = MESES_ABREV[MES_PROCESSAR]
        df_mes = df[df['aniversario_empresa MÊS'].str.strip().str.capitalize() == mes_abrev]

        if df_mes.empty:
            print(f"[INFO] Não há aniversariantes de empresa no mês {mes_abrev}")
            return

        nomes = df_mes['Nome Funcionário'].tolist()
        tempos = df_mes['Tempo Empresa'].tolist()

        if not nomes:
            print("[INFO] Nenhum colaborador encontrado para o mês.")
            return

        outlook = win32com.client.Dispatch("Outlook.Application")
        arquivos_cartoes = os.listdir(PASTA_CARDS) if os.path.exists(PASTA_CARDS) else []

        # Montagem dos nomes formatados
        def formatar_tempo(t):
            return f"{t} ano" if t == 1 else f"{t} anos"

        nomes_txt = "\n".join(f"{nome}, {formatar_tempo(tempo)};" for nome, tempo in zip(nomes, tempos))
        nomes_html = "<br>".join(f"<b>{nome}</b>, {formatar_tempo(tempo)};" for nome, tempo in zip(nomes, tempos))

        data_evento = datetime(ANO, MES_PROCESSAR, 1, 8, 0)

        print(f"\n[PROCESSANDO] Aniversário de empresa para {nomes_txt}")

        # === Criar Evento (sem imagem da marca) ===
        appointment = outlook.CreateItem(1)

        if len(nomes) == 1:
            titulo = f"Aniversário de Empresa: {nomes[0]}"
        else:
            titulo = f"Aniversário de Empresa dos Colaboradores"

        appointment.Subject = titulo
        appointment.Start = data_evento
        appointment.Duration = 30
        appointment.AllDayEvent = False
        appointment.ReminderSet = True
        appointment.ReminderMinutesBeforeStart = 1440
        appointment.BusyStatus = 0

        corpo_evento = f"""Olá!
Segue lembrete de aniversário de empresa para os colaboradores:
{nomes_txt}

ATENÇÃO!: No dia do aniversário de empresa dos referidos colaboradores deve-se enviar a imagem em anexo na equipe de celebrações no Teams.

Atenciosamente,
"""
        appointment.Body = corpo_evento

        # Anexar cartões no evento
        anexados = 0
        for nome in nomes:
            nome_lower = nome.lower()
            for arquivo in arquivos_cartoes:
                if nome_lower in arquivo.lower():
                    caminho_cartao = os.path.join(PASTA_CARDS, arquivo)
                    appointment.Attachments.Add(caminho_cartao)
                    anexados += 1
                    break
            else:
                print(f"[AVISO] Cartão não encontrado para {nome} (evento)")

        print(f"[INFO] {anexados} cartões anexados ao evento.")

        appointment.Display()  # mostra para revisão

        # === Criar Email (com imagem da marca inline) ===
        mail = outlook.CreateItem(0)
        mail.Subject = f"Lembrete de Aniversário de Empresa - {data_evento.strftime('%d/%m/%Y')}"
        mail.To = EMAIL_DESTINO

        # Anexar cartões no email
        anexados_email = 0
        for nome in nomes:
            nome_lower = nome.lower()
            for arquivo in arquivos_cartoes:
                if nome_lower in arquivo.lower():
                    caminho_cartao = os.path.join(PASTA_CARDS, arquivo)
                    mail.Attachments.Add(caminho_cartao)
                    anexados_email += 1
                    break
            else:
                print(f"[AVISO] Cartão não encontrado para {nome} (email)")

        # Adicionar imagem da marca inline (somente no e-mail)
        img_tag = ""
        if os.path.exists(CAMINHO_MARCA):
            attachment = mail.Attachments.Add(CAMINHO_MARCA)
            attachment.PropertyAccessor.SetProperty(
                "http://schemas.microsoft.com/mapi/proptag/0x3712001F", "marca"
            )
            img_tag = '<p><img src="cid:marca" style="width:300px;"></p>'

        corpo_email = f"""
<html>
<body>
<p>Olá!</p>

<p>Segue lembrete de aniversário de empresa para os colaboradores:<br>
{nomes_html}</p>

<p><b>ATENÇÃO!:</b> No dia do aniversário de empresa dos referidos colaboradores deve-se enviar a imagem em anexo na equipe de celebrações no Teams.</p>

<p>Atenciosamente,</p>

{img_tag}

</body>
</html>
"""
        mail.HTMLBody = corpo_email

        print(f"[INFO] {anexados_email} cartões anexados ao email.")

        mail.Display()  # mostra para revisão

        print(f"[OK] Evento e email preparados para {nomes_txt}")
        print("\n[FINALIZADO] Processamento concluído!")

    except Exception as e:
        print(f"[ERRO] Falha geral na automação: {str(e)}")


if __name__ == "__main__":
    processar_aniversario_empresa()


[ERRO] Falha geral na automação: [Errno 2] No such file or directory: 'C:\\Users\\lz6321\\OneDrive - grendene.com.br\\Documentos\\automation\\Automação_Aniversariantes\\Aniversariante - Empresa.xlsx'
